In [0]:
%sh
pip install --upgrade pip
pip install bs4
pip install nltk
pip install pyspellchecker
pip install python-Levenshtein

Requirement already satisfied: pip in /databricks/python3/lib/python3.8/site-packages (21.3.1)
Requirement already satisfied: bs4 in /databricks/python3/lib/python3.8/site-packages (0.0.1)
Requirement already satisfied: beautifulsoup4 in /databricks/python3/lib/python3.8/site-packages (from bs4) (4.10.0)
Requirement already satisfied: soupsieve>1.2 in /databricks/python3/lib/python3.8/site-packages (from beautifulsoup4->bs4) (2.3)
Requirement already satisfied: nltk in /databricks/python3/lib/python3.8/site-packages (3.6.5)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (0.17.0)
Requirement already satisfied: regex>=2021.8.3 in /databricks/python3/lib/python3.8/site-packages (from nltk) (2021.11.2)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from nltk) (4.62.3)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from nltk) (8.0.3)
Requirement already satisfied: pyspellchecker in /databricks/python3/lib/python3.8/site-packages (0.6.2)
Requirement already satisfied: python-Levenshtein in /databricks/python3/lib/python3.8/site-packages (0.12.2)
Requirement already satisfied: setuptools in /databricks/python3/lib/python3.8/site-packages (from python-Levenshtein) (50.3.1)

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
Out[91]: True

## Answer to Question 1
“this is jsut graet!”
1. How can you fix typos in the above sentence using Levenshtein distance algorithm? (write a small python code to show)<br>

  Using Levenshtein distance algorithm can fix typos by indicating the minimum number of operations (insert, delete and replace) that needs to be applied to a misspelled word in comparison to a correct spelled word. We may find the lev distance between the misspelled word and the correct word and then applied number of operations to fix the typo. See code below.

2. For each typo, what kind of edit operation you need to do? (consider the three edit operations: insert, delete, replace)<br>
There are several ways to apply edit operation to fix the typo.<br>
  For typo 'jsut': <br>
    - Method 1: we can first delete the letter `s` and then insert a letter `s` after the letter `u`.
    - Method 2: we can first replace the letter `s` with a letter `u` and then replace the next letter `u` with a letter `s`

  For typo 'graet': <br>
    - Method 1: we can first delete the letter `a` and then insert a letter `a` after the letter `e`.
    - Method 2: we can first replace the letter `a` with a letter `e` and then replace the next letter `e` with a letter `a`

In [0]:
import Levenshtein as lev
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

spell = SpellChecker()
    
text = "this is jsut graet!"
words = word_tokenize(text)

# There are some limitations in spellchecker module which identifies the word 'jsut'
# as a correct word. Hence, I had to remove it from its list of word from the module
# to apply and fix the typo.
spell.word_frequency.remove('jsut') 

misspelled = spell.unknown(words)
for word in misspelled:
  print("Typo = %s. Correct word = %s. Levenshtein Distance = %d"
        %(word, spell.correction(word), lev.distance(word, spell.correction(word))))

Typo = jsut. Correct word = just. Levenshtein Distance = 2
Typo = graet. Correct word = great. Levenshtein Distance = 2

In [0]:
from nltk.corpus import stopwords
stop_en = stopwords.words('english')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
sb = SnowballStemmer(language = 'english')

from bs4 import BeautifulSoup
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import lit
from pyspark.sql.functions import sqrt
from pyspark.sql import Window
import pyspark.sql.types as t

import math

## Answer to Question 2

In [0]:
@udf
def preprocess_udf(body):
    body = BeautifulSoup(body)
    #removed noise
    urls  =  body.find_all('a')
    if len(urls) > 0: body.a.clear()
    #removed noise
    codes = body.find_all('code')
    if len(codes) > 0: body.code.clear()
    #removed noise
    pres = body.find_all('pre')
    if len(pres) > 0: body.pre.clear()
    text = body.get_text()
    words = []
    text = text.lower() #convert words to lowercase
    sents = nltk.sent_tokenize(text)
    for sent in sents:
      for word in nltk.word_tokenize(sent):
        if word in stop_en: continue #removed stopwords
        if len(word) < 3: continue #removed words with less than 3 characters
        words.append(sb.stem(word))#transform word to root
    return " ".join(words)

def distinct(text):
    unique = []
    seen = set()
    for word in text.split():
        if word not in seen: #removed duplicate words
            unique.append(word)
            seen.add(word)
    return unique
unique = f.udf(distinct, t.ArrayType(t.StringType()))

#verify length and reduction of words/characters per every step
# slen = udf(lambda s: len(s), t.IntegerType())

def dtm(text, compare):
    d = [0] * len(compare)
    for word in text.split():
      if word in compare:
        d[compare.index(word)] += 1
    return d
occurrences = f.udf(dtm, t.ArrayType(t.IntegerType()))

def cosine_similarity(dt1, dt2):
    sum_total = 0;
    sum_dt1 = 0;
    sum_dt2 = 0;
    for x,y in zip(dt1, dt2):
      sum_total += x * y
      sum_dt1 += x * x
      sum_dt2 += y * y
    similar = sum_total / math.sqrt(sum_dt1 * sum_dt2)
    return similar
similarity = f.udf(cosine_similarity, t.DoubleType())

class SOAnalysis(object):
  
  def __init__(self, progLang):
    pass

  def getFile(self):
    file_location = "/FileStore/tables/SO_"+progLang+".csv"
    self.df = spark.read.option("header", True).option("wholeFile", True).option("escape", "\"").option("multiLine", True).csv(file_location)
      
  def filtered_cleanDF(self):
    df0 = self.df.select('Title', preprocess_udf("Title").alias("title_cleaned"),
                         'Body', preprocess_udf("Body").alias("body_cleaned"),
                         'AcceptedAnswerId', 'Score').drop("Title").drop("Body")
    df0 = df0.withColumn("TextualContent", concat_ws(' ', df0.title_cleaned, df0.body_cleaned))
    df0 = df0.drop("title_cleaned").drop("body_cleaned")
    return df0
  
  def getMaxScoreUnacceptedAns(self):
    unaccepted_df = self.filtered_cleanDF().where('TextualContent is not null').where('AcceptedAnswerId is null')
    unaccepted_df = unaccepted_df.withColumn("Score", unaccepted_df["Score"].cast(t.IntegerType()))
    w = Window.partitionBy('AcceptedAnswerId')
    unaccepted_df = unaccepted_df.withColumn('maxScore', f.max('Score').over(w)).where(f.col('Score') == f.col('maxScore')).drop('maxScore')
    return unaccepted_df

  def similarityAnalysis(self):
    UnacceptedQuestion = self.getMaxScoreUnacceptedAns().select('TextualContent').first()['TextualContent']
    df1 = self.filtered_cleanDF().where('TextualContent is not null').where('AcceptedAnswerId is not null')
    df2 = df1.withColumn("UnacceptedQuestion", lit(UnacceptedQuestion))
    df2 = df2.withColumn("Merged_Comparison", concat_ws(' ', df2.TextualContent, df2.UnacceptedQuestion))
    df3 = df2.select('*', unique("Merged_Comparison").alias("DistinctWords")).drop("Merged_Comparison")
    df4 = df3.select('*', occurrences("TextualContent","DistinctWords").alias("DT1"),
                     occurrences("UnacceptedQuestion","DistinctWords").alias("DT2"))
    df5 = df4.select('*', similarity("DT1","DT2").alias("CosineSimilarity")).drop("DistinctWords")
    return df5
  
  def getTopSimilarQuestions(self):
    return self.similarityAnalysis().orderBy(['CosineSimilarity'], ascending=[0])

In [0]:
progLang = 'Python'
soAnalysis = SOAnalysis(progLang)
soAnalysis.getFile()
soAnalysis.filtered_cleanDF().show()

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 null| 25|pros con differ a...|
 43794| 18|modulus oper nega...|
 null| 0|python win32com l...|
 null| 6|hide user name pa...|
 62428832| 32|anaconda python c...|
 16212890| -1|python reg exp de...|
 16216772| -2|pexpect prompt bl...|
 null| 0|gui applic run sp...|
 48306460| 0|cant't get recurs...|
 null| 4|fluentd post stdo...|
 48304849| 0|panda creat new c...|
 48304390| 2|implicit relat im...|
 null| 1|slice list withou...|
 null| 1|creat list group ...|
 48305090| 0|replac doubl loop...|
 null| 2|iter text file fi...|
 null| 1|deal list messag ...|
 null| 0|python ad diction...|
 null| 10|invis watermark i...|
 135468| 41|project design la...|
+----------------+-----+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getMaxScoreUnacceptedAns().printSchema()
soAnalysis.getMaxScoreUnacceptedAns().show()

root
-- AcceptedAnswerId: string (nullable = true)
-- Score: integer (nullable = true)
-- TextualContent: string (nullable = false)

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 null| 1757|limit float two d...|
+----------------+-----+--------------------+

In [0]:
soAnalysis.similarityAnalysis().printSchema()
soAnalysis.similarityAnalysis().show()

root
-- AcceptedAnswerId: string (nullable = true)
-- Score: string (nullable = true)
-- TextualContent: string (nullable = false)
-- UnacceptedQuestion: string (nullable = false)
-- DT1: array (nullable = true)
 |-- element: integer (containsNull = true)
-- DT2: array (nullable = true)
 |-- element: integer (containsNull = true)
-- CosineSimilarity: double (nullable = true)

+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 43794| 18|modulus oper nega...|limit float two d...|[1, 1, 1, 1, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 62428832| 32|anaconda python c...|limit float two d...|[4, 1, 3, 2, 2, 2...|[0, 0, 0, 0, 0, 0...| 0.08164965809277261|
 16212890| -1|python reg exp de...|limit float two d...|[1, 1, 1, 1, 1, 1...|[0, 0, 0, 0, 0, 0...|0.040422604172722164|
 16216772| -2|pexpect prompt bl...|limit float two d...|[2, 4, 3, 3, 2, 1...|[0, 0, 0, 0, 0, 0...| 0.08333333333333333|
 48306460| 0|cant't get recurs...|limit float two d...|[1, 2, 2, 1, 3, 3...|[0, 0, 0, 0, 0, 0...| 0.07509392614826384|
 48304849| 0|panda creat new c...|limit float two d...|[1, 2, 3, 4, 6, 2...|[0, 0, 0, 0, 0, 0...|0.021124141684149264|
 48304390| 2|implicit relat im...|limit float two d...|[1, 1, 4, 2, 2, 2...|[0, 0, 0, 1, 0, 0...| 0.03513641844631533|
 48305090| 0|replac doubl loop...|limit float two d...|[1, 2, 3, 1, 2, 1...|[0, 0, 0, 0, 0, 0...| 0.03908317935314764|
 135468| 41|project design la...|limit float two d...|[7, 1, 3, 4, 2, 1...|[0, 0, 0, 0, 0, 0...|0.025031308716087945|
 48305919| 0|authent googl sto...|limit float two d...|[2, 1, 1, 1, 2, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 48305961| 0|panda append conc...|limit float two d...|[2, 2, 1, 2, 5, 1...|[0, 0, 0, 0, 0, 0...|0.028867513459481284|
 48310968| -1|get python 2to3 l...|limit float two d...|[2, 3, 1, 2, 3, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 48306348| -4|evalu undefin var...|limit float two d...|[2, 2, 2, 3, 3, 1...|[0, 0, 0, 0, 0, 0...| 0.03790490217894517|
 40296279| 0|arbitrari byte va...|limit float two d...|[1, 1, 1, 3, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.08512565307587486|
 48607426| 1|nvme throughput t...|limit float two d...|[3, 4, 4, 6, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.03450327796711771|
 24551140| 3|two correspond y-...|limit float two d...|[3, 3, 4, 2, 1, 4...|[1, 0, 0, 0, 0, 0...| 0.16933350266692068|
 3050687| 12|possibl use multi...|limit float two d...|[2, 3, 1, 1, 1, 3...|[0, 0, 0, 0, 0, 0...| 0.13333333333333333|
 40513160| 1|python find index...|limit float two d...|[1, 2, 3, 2, 2, 2...|[0, 0, 0, 0, 0, 0...| 0.0|
 43125201| 0|jqueri modal conf...|limit float two d...|[1, 6, 2, 2, 13, ...|[0, 0, 0, 0, 0, 0...| 0.05284573803625948|
 16212697| 0|extract inform js...|limit float two d...|[3, 3, 3, 2, 3, 2...|[0, 0, 0, 0, 0, 0...|0.028444006199428714|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getTopSimilarQuestions().show(3)

+----------------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+
 56672970| 0|round big float p...|limit float two d...|[3, 1, 2, 1, 2, 1...|[1, 0, 1, 1, 0, 0...|0.39605901719066977|
 1598650| 56|round decim new p...|limit float two d...|[2, 3, 1, 2, 1, 2...|[1, 1, 0, 0, 0, 1...| 0.3818813079129867|
 8869156| 22|problem round dec...|limit float two d...|[2, 7, 13, 8, 2, ...|[0, 1, 1, 0, 0, 0...| 0.380920029682232|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+
only showing top 3 rows

In [0]:
progLang = 'Java'
soAnalysis = SOAnalysis(progLang)
soAnalysis.getFile()
soAnalysis.filtered_cleanDF().show()

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 null| 0|sort collect use ...|
 null| 1|java array list g...|
 24361056| 2|spring aop reentr...|
 43807| 63|convert date stri...|
 43830| 6|prefetch oracl se...|
 48304412| 2|extrem slow files...|
 null| 7|creat button pdf ...|
 null| 5|gmail api servic ...|
 null| 1|spring boot fail ...|
 null| 0|java librari forw...|
 8191256| 2|handl xml jaxb an...|
 null| 0|quartz job schedu...|
 48381679| 0|replac locat head...|
 null| 1|display one div t...|
 null| 0|java netbean give...|
 null| 1|want print succes...|
 null| -2|n't find reason f...|
 null| 0|upload imag djang...|
 17705336| 0|append xml elemen...|
 17701237| 1|remov duplic obje...|
+----------------+-----+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getMaxScoreUnacceptedAns().show()

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 null| 1668|sort map valu rel...|
+----------------+-----+--------------------+

In [0]:
soAnalysis.similarityAnalysis().show()

+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 24361056| 2|spring aop reentr...|sort map valu rel...|[2, 2, 2, 4, 1, 1...|[0, 0, 0, 0, 0, 0...|0.012262786789699318|
 43807| 63|convert date stri...|sort map valu rel...|[1, 5, 1, 3, 2, 1...|[1, 0, 0, 0, 0, 0...|0.045175395145262566|
 43830| 6|prefetch oracl se...|sort map valu rel...|[3, 2, 4, 1, 2, 3...|[0, 0, 0, 0, 0, 0...|0.058338335119694794|
 48304412| 2|extrem slow files...|sort map valu rel...|[2, 2, 1, 1, 3, 1...|[0, 0, 0, 0, 0, 0...|0.025058800551629404|
 8191256| 2|handl xml jaxb an...|sort map valu rel...|[1, 2, 2, 2, 1, 2...|[0, 0, 0, 0, 0, 0...| 0.07941225539113157|
 48381679| 0|replac locat head...|sort map valu rel...|[4, 6, 5, 4, 1, 1...|[0, 0, 0, 0, 0, 0...|0.030297814186234104|
 17705336| 0|append xml elemen...|sort map valu rel...|[2, 4, 2, 2, 2, 3...|[0, 0, 0, 0, 0, 0...|0.026894744944055273|
 17701237| 1|remov duplic obje...|sort map valu rel...|[2, 3, 2, 1, 6, 1...|[0, 0, 0, 0, 0, 0...| 0.09824718648648241|
 41703975| 2|execut testng.xml...|sort map valu rel...|[1, 2, 1, 1, 2, 1...|[0, 0, 0, 0, 0, 0...|0.025482359571881278|
 3050687| 12|possibl use multi...|sort map valu rel...|[2, 3, 1, 1, 1, 3...|[0, 0, 0, 0, 0, 0...|0.027602622373694166|
 3050804| 1|covari java follo...|sort map valu rel...|[2, 3, 1, 2, 1, 1...|[0, 1, 0, 0, 0, 0...| 0.13944333775567927|
 24325765| 0|take rate valu us...|sort map valu rel...|[2, 3, 2, 3, 2, 2...|[0, 0, 4, 0, 0, 0...| 0.1267731382092775|
 24331512| 1|differ use opengl...|sort map valu rel...|[3, 3, 3, 3, 2, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 24325364| 3|unknown tag undef...|sort map valu rel...|[2, 2, 3, 3, 2, 1...|[0, 0, 0, 0, 0, 0...|0.021731429260625337|
 16212874| 1|perform java code...|sort map valu rel...|[1, 1, 3, 1, 2, 1...|[0, 1, 0, 0, 0, 0...| 0.08530901495354998|
 16212806| 0|troubl return obj...|sort map valu rel...|[1, 4, 4, 15, 2, ...|[0, 0, 0, 0, 0, 1...| 0.04197177036168771|
 40295145| -1|random number num...|sort map valu rel...|[5, 6, 3, 1, 2, 5...|[0, 0, 0, 0, 0, 0...| 0.03624401881463749|
 43852| 6|would programmat ...|sort map valu rel...|[2, 1, 2, 4, 4, 1...|[0, 0, 0, 0, 0, 0...|0.045175395145262566|
 3050911| 13|virtual function ...|sort map valu rel...|[5, 6, 2, 2, 1, 1...|[0, 0, 0, 0, 0, 0...|0.011952286093343936|
 45989| 5|start learn java ...|sort map valu rel...|[2, 1, 3, 1, 2, 1...|[0, 0, 1, 0, 0, 0...| 0.07559289460184544|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getTopSimilarQuestions().show(3)

+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
 32078348| 0|java array sort n...|sort map valu rel...|[1, 2, 2, 1, 1, 1...|[1, 3, 5, 1, 0, 0...|0.6377481392176932|
 56559281| 0|sort map valu jav...|sort map valu rel...|[3, 3, 4, 1, 1, 2...|[5, 1, 4, 1, 0, 0...|0.6236095644623235|
 9168393| 3|sort hashmap valu...|sort map valu rel...|[5, 3, 4, 1, 1, 1...|[5, 0, 4, 1, 0, 0...|0.6232538633263821|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
only showing top 3 rows

In [0]:
progLang = 'Javascript'
soAnalysis = SOAnalysis(progLang)
soAnalysis.getFile()
soAnalysis.filtered_cleanDF().show()

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 9753499| 3|confus hoist cons...|
 9753398| 1|use closur simpl ...|
 null| 1|circular hour sel...|
 null| 0|kendoui tabstrip ...|
 17702613| 0|chang background ...|
 null| 0|display popup win...|
 16213296| 0|merg join regular...|
 16527927| 1|jqueri imag slide...|
 null| 1|hiding/dis submit...|
 40426006| 0|tri access ej arr...|
 null| 1|dynam set form na...|
 null| 0|display last numb...|
 2859898| 3|googl chrome exte...|
 null| 3|download comput v...|
 3050818| 0|code move previou...|
 16215302| -1|write html css ja...|
 16216682| 0|knockout.j use fo...|
 47168| 9|javascript css pa...|
 40304380| 2|tri chang datavie...|
 40295170| 0|apollo server rea...|
+----------------+-----+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getMaxScoreUnacceptedAns().show()

+----------------+-----+--------------------+
AcceptedAnswerId|Score| TextualContent|
+----------------+-----+--------------------+
 null| 808|make ajax call wi...|
+----------------+-----+--------------------+

In [0]:
soAnalysis.similarityAnalysis().show()

+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 9753499| 3|confus hoist cons...|make ajax call wi...|[1, 2, 1, 1, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 9753398| 1|use closur simpl ...|make ajax call wi...|[1, 1, 1, 2, 1, 1...|[2, 0, 0, 0, 0, 0...| 0.05773502691896257|
 17702613| 0|chang background ...|make ajax call wi...|[3, 3, 3, 3, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 16213296| 0|merg join regular...|make ajax call wi...|[1, 1, 2, 2, 7, 2...|[0, 0, 0, 0, 0, 0...| 0.07882634225314346|
 16527927| 1|jqueri imag slide...|make ajax call wi...|[1, 2, 2, 3, 2, 3...|[2, 0, 0, 0, 0, 0...| 0.0917662935482247|
 40426006| 0|tri access ej arr...|make ajax call wi...|[1, 2, 2, 3, 3, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 2859898| 3|googl chrome exte...|make ajax call wi...|[1, 1, 1, 5, 2, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 3050818| 0|code move previou...|make ajax call wi...|[3, 1, 2, 3, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 16215302| -1|write html css ja...|make ajax call wi...|[2, 2, 2, 2, 3, 4...|[0, 0, 0, 1, 0, 0...|0.047809144373375745|
 16216682| 0|knockout.j use fo...|make ajax call wi...|[1, 1, 3, 1, 1, 1...|[0, 2, 0, 0, 0, 0...|0.022755204959542974|
 47168| 9|javascript css pa...|make ajax call wi...|[5, 5, 2, 3, 2, 1...|[1, 0, 0, 0, 0, 0...| 0.10583005244258362|
 40304380| 2|tri chang datavie...|make ajax call wi...|[2, 3, 2, 2, 1, 3...|[0, 0, 0, 0, 0, 0...|0.029568850838182913|
 40295170| 0|apollo server rea...|make ajax call wi...|[2, 1, 3, 3, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.0|
 16575647| 29|get offset posit ...|make ajax call wi...|[3, 5, 6, 5, 3, 2...|[0, 0, 0, 0, 0, 0...| 0.03142696805273545|
 16212945| 1|variabl return na...|make ajax call wi...|[2, 1, 2, 1, 1, 1...|[0, 0, 0, 0, 0, 0...| 0.03577708763999664|
 8189923| -1|javascript class ...|make ajax call wi...|[2, 2, 2, 2, 1, 1...|[1, 0, 0, 0, 0, 0...| 0.0834057656228299|
 48305956| 3|write .d.ts file ...|make ajax call wi...|[3, 2, 9, 1, 6, 3...|[0, 0, 0, 0, 0, 1...| 0.11759494644146673|
 40295756| 0|wont radio button...|make ajax call wi...|[1, 4, 3, 1, 1, 2...|[0, 0, 0, 0, 0, 0...|0.029346959282671108|
 24325231| 0|direct work selec...|make ajax call wi...|[3, 2, 2, 1, 7, 2...|[0, 0, 0, 0, 0, 0...| 0.0|
 43824| 7|jqueri slice clic...|make ajax call wi...|[2, 1, 2, 1, 1, 1...|[2, 0, 0, 0, 0, 0...| 0.09701425001453319|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
soAnalysis.getTopSimilarQuestions().show(3)

+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
AcceptedAnswerId|Score| TextualContent| UnacceptedQuestion| DT1| DT2| CosineSimilarity|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
 533886| 0|javascript functi...|make ajax call wi...|[4, 2, 2, 4, 2, 1...|[1, 0, 0, 2, 2, 0...|0.5633622717047423|
 24804061| -1|show/ hide div de...|make ajax call wi...|[1, 1, 2, 2, 2, 2...|[0, 0, 0, 0, 0, 0...|0.5405899027195888|
 10422368| 0|serial valu witho...|make ajax call wi...|[2, 1, 1, 2, 1, 1...|[0, 0, 2, 0, 0, 0...|0.5393598899705937|
+----------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
only showing top 3 rows